In [2]:
import numpy as np
from tabulate import tabulate

# Set random seed for reproducibility
np.random.seed(42)

# Generate our sample data
temperature = np.array([18.2, 19.5, 20.1, 22.4, 23.8, 25.0, 23.2, 21.5, 19.8, 17.5])
humidity = np.array([65.2, 62.8, 58.5, 55.0, 45.2, 42.1, 48.5, 52.3, 60.5, 67.8])
wind_speed = np.array([5.2, 6.8, 8.5, 10.2, 12.5, 14.8, 13.2, 11.5, 9.2, 6.5])
X = np.column_stack((temperature, humidity, wind_speed))

# Target: power consumption (kWh)
y = 2.5 * temperature - 0.5 * humidity + 1.2 * wind_speed + np.random.normal(0, 5, 10)

# Create sequences with lookback of 2 (predict the step after the sequence)
def create_sequences(X, y, lookback=2):
    X_seq, y_seq = [], []
    for i in range(len(X) - lookback):
        X_seq.append(X[i:i+lookback])
        y_seq.append(y[i+lookback])
    return np.array(X_seq), np.array(y_seq)

X_sequences, y_sequences = create_sequences(X, y, lookback=2)

# Define LSTM dimensions
n_features = 3  # Temperature, humidity, wind speed
n_hidden = 4    # Number of LSTM units
n_output = 1    # Power consumption prediction

# Initialize weight matrices and biases
# For simplicity, we'll use small random values
W_f = np.random.randn(n_hidden, n_features + n_hidden) * 0.01
b_f = np.zeros(n_hidden)

W_i = np.random.randn(n_hidden, n_features + n_hidden) * 0.01
b_i = np.zeros(n_hidden)

W_c = np.random.randn(n_hidden, n_features + n_hidden) * 0.01
b_c = np.zeros(n_hidden)

W_o = np.random.randn(n_hidden, n_features + n_hidden) * 0.01
b_o = np.zeros(n_hidden)

W_y = np.random.randn(n_output, n_hidden) * 0.01
b_y = np.zeros(n_output)

# Keep copies of initial weights for comparison
W_f_initial = W_f.copy()
W_i_initial = W_i.copy() 
W_c_initial = W_c.copy()
W_o_initial = W_o.copy()
W_y_initial = W_y.copy()

# Define activation functions and their derivatives
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def tanh(x):
    return np.tanh(x)

def sigmoid_derivative(values):
    return values * (1 - values)

def tanh_derivative(values):
    return 1 - values**2

# Loss function (MSE) and its derivative
def mse_loss(y_pred, y_true):
    return np.mean((y_pred - y_true)**2)

# Loss function (MSE) and its derivative
def mse_loss(y_pred, y_true):
    return np.mean((y_pred - y_true)**2)

def mse_derivative(y_pred, y_true):
    # Handle both scalar and array cases
    if np.isscalar(y_true) or (hasattr(y_true, 'shape') and len(y_true.shape) == 0):
        return 2 * (y_pred - y_true)
    else:
        return 2 * (y_pred - y_true) / len(y_true)

# Learning rate
learning_rate = 0.01

# Track metrics across epochs
epoch_losses = []
weight_changes = []

print("=" * 80)
print("LSTM BACKPROPAGATION THROUGH TIME (BPTT) DEMONSTRATION")
print("=" * 80)
print(f"Training on {len(X_sequences)} sequences with lookback=2 for 3 epochs")
print(f"Learning rate: {learning_rate}")
print("=" * 80)

for epoch in range(3):
    print(f"\n{'=' * 30} EPOCH {epoch+1} {'=' * 30}")
    
    # Initialize states for this epoch
    h_t = np.zeros(n_hidden)
    C_t = np.zeros(n_hidden)
    
    epoch_predictions = []
    epoch_true_values = []
    
    # Forward pass: Store all intermediate values needed for backpropagation
    forward_states = []
    
    for seq_idx, sequence in enumerate(X_sequences):
        print(f"\n--- FORWARD PASS: Sequence {seq_idx+1} (Days {seq_idx+1}-{seq_idx+2}) ---")
        
        # Store all intermediate values for this sequence
        sequence_cache = {'x': [], 'combined': [], 'f_gate': [], 'i_gate': [], 
                         'c_tilde': [], 'o_gate': [], 'c_state': [], 'h_state': []}
        
        # Store initial states for this sequence
        h_prev = h_t.copy()
        C_prev = C_t.copy()
        sequence_cache['h_state'].append(h_prev)
        sequence_cache['c_state'].append(C_prev)
        
        # Process each time step in the sequence (forward pass)
        for t in range(len(sequence)):
            x_t = sequence[t]
            day_num = seq_idx + t + 1
            
            # Store input
            sequence_cache['x'].append(x_t)
            
            # Concatenate input with previous hidden state
            combined = np.concatenate([x_t, h_t])
            sequence_cache['combined'].append(combined)
            
            # Calculate gate activations
            f_t = sigmoid(np.dot(W_f, combined) + b_f)
            i_t = sigmoid(np.dot(W_i, combined) + b_i)
            c_tilde = tanh(np.dot(W_c, combined) + b_c)
            o_t = sigmoid(np.dot(W_o, combined) + b_o)
            
            # Store gate values
            sequence_cache['f_gate'].append(f_t)
            sequence_cache['i_gate'].append(i_t)
            sequence_cache['c_tilde'].append(c_tilde)
            sequence_cache['o_gate'].append(o_t)
            
            # Update cell state
            C_t = f_t * C_t + i_t * c_tilde
            
            # Update hidden state
            h_t = o_t * tanh(C_t)
            
            # Store updated states
            sequence_cache['c_state'].append(C_t.copy())
            sequence_cache['h_state'].append(h_t.copy())
            
            print(f"  Processed Day {day_num} - Hidden state norm: {np.linalg.norm(h_t):.4f}")
        
        # Make prediction for this sequence
        y_pred = np.dot(W_y, h_t) + b_y
        
        # Store prediction and actual value
        epoch_predictions.append(y_pred[0])
        epoch_true_values.append(y_sequences[seq_idx])
        
        # Store for backpropagation
        sequence_cache['y_pred'] = y_pred
        sequence_cache['y_true'] = y_sequences[seq_idx]
        
        print(f"  Prediction for Day {seq_idx+3}: {y_pred[0]:.4f} kWh (Actual: {y_sequences[seq_idx]:.4f} kWh)")
        
        # Store all cache for this sequence
        forward_states.append(sequence_cache)
    
    # Calculate epoch loss
    epoch_loss = mse_loss(np.array(epoch_predictions), y_sequences)
    epoch_losses.append(epoch_loss)
    print(f"\nEpoch {epoch+1} Loss: {epoch_loss:.4f}")
    
    print("\n--- BACKPROPAGATION THROUGH TIME ---")
    
    # Initialize gradients
    dW_f = np.zeros_like(W_f)
    db_f = np.zeros_like(b_f)
    dW_i = np.zeros_like(W_i)
    db_i = np.zeros_like(b_i)
    dW_c = np.zeros_like(W_c)
    db_c = np.zeros_like(b_c)
    dW_o = np.zeros_like(W_o)
    db_o = np.zeros_like(b_o)
    dW_y = np.zeros_like(W_y)
    db_y = np.zeros_like(b_y)
    
    # Process sequences in reverse order for BPTT
    for seq_idx in range(len(forward_states)-1, -1, -1):
        cache = forward_states[seq_idx]
        
        print(f"\n  Backpropagating Sequence {seq_idx+1}")
        
        # Initialize gradient of the loss with respect to the output
        dy = mse_derivative(cache['y_pred'], cache['y_true'])  # Shape: (1,)
        
        # Gradient for the output layer
        dW_y += np.outer(dy, cache['h_state'][-1])  # Gradient for W_y
        db_y += dy  # Gradient for b_y
        
        # Initial gradient with respect to hidden state
        dh_next = np.dot(W_y.T, dy)  # Shape: (n_hidden,)
        dC_next = np.zeros_like(cache['c_state'][0])  # Shape: (n_hidden,)
        
        # Backpropagate through time steps in reverse order
        for t in range(len(cache['x'])-1, -1, -1):
            # Current hidden state and cell state
            h_t = cache['h_state'][t+1]  # +1 because we stored initial state at index 0
            C_t = cache['c_state'][t+1]
            C_prev = cache['c_state'][t]
            
            # Gate values
            o_t = cache['o_gate'][t]
            f_t = cache['f_gate'][t]
            i_t = cache['i_gate'][t]
            c_tilde = cache['c_tilde'][t]
            
            # Backpropagate through the tanh in h_t = o_t * tanh(C_t)
            dC_part = dh_next * o_t * tanh_derivative(tanh(C_t))
            dC = dC_next + dC_part
            
            # Backpropagate through the gates
            do = dh_next * tanh(C_t) * sigmoid_derivative(o_t)
            df = dC * C_prev * sigmoid_derivative(f_t)
            di = dC * c_tilde * sigmoid_derivative(i_t)
            dc_tilde = dC * i_t * tanh_derivative(c_tilde)
            
            # Combined input
            combined = cache['combined'][t]
            
            # Gradient for gate weights and biases
            dW_f += np.outer(df, combined)
            db_f += df
            dW_i += np.outer(di, combined)
            db_i += di
            dW_c += np.outer(dc_tilde, combined)
            db_c += dc_tilde
            dW_o += np.outer(do, combined)
            db_o += do
            
            # Gradient with respect to the combined input
            dcombined = (np.dot(W_f.T, df) + 
                        np.dot(W_i.T, di) + 
                        np.dot(W_c.T, dc_tilde) + 
                        np.dot(W_o.T, do))
            
            # Split gradient between x and h components
            dx = dcombined[:n_features]
            dh_prev = dcombined[n_features:]
            
            # Gradient for next backward step
            dh_next = dh_prev
            dC_next = dC * f_t
            
            print(f"    Backpropagated timestep {t+1} - Gradient norm: {np.linalg.norm(dh_next):.4f}")
    
    # Apply gradients with clipping
    clip_threshold = 5.0
    
    # Function to clip gradients
    def clip_gradient(grad):
        norm = np.linalg.norm(grad)
        if norm > clip_threshold:
            return grad * (clip_threshold / norm)
        return grad
    
    # Clip and apply gradients
    dW_f = clip_gradient(dW_f)
    dW_i = clip_gradient(dW_i)
    dW_c = clip_gradient(dW_c)
    dW_o = clip_gradient(dW_o)
    dW_y = clip_gradient(dW_y)
    
    # Record weight changes for this epoch
    weight_changes_epoch = {
        'W_f': np.linalg.norm(dW_f * learning_rate),
        'W_i': np.linalg.norm(dW_i * learning_rate),
        'W_c': np.linalg.norm(dW_c * learning_rate),
        'W_o': np.linalg.norm(dW_o * learning_rate),
        'W_y': np.linalg.norm(dW_y * learning_rate)
    }
    weight_changes.append(weight_changes_epoch)
    
    # Update weights
    W_f -= learning_rate * dW_f
    b_f -= learning_rate * db_f
    W_i -= learning_rate * dW_i
    b_i -= learning_rate * db_i
    W_c -= learning_rate * dW_c
    b_c -= learning_rate * db_c
    W_o -= learning_rate * dW_o
    b_o -= learning_rate * db_o
    W_y -= learning_rate * dW_y
    b_y -= learning_rate * db_y
    
    print(f"\n  Applied gradients with learning rate {learning_rate} and clipping threshold {clip_threshold}")
    
    # Print weight update summary
    print("\n  Weight Update Summary:")
    weight_table = [
        ["W_f", f"{weight_changes_epoch['W_f']:.6f}"],
        ["W_i", f"{weight_changes_epoch['W_i']:.6f}"],
        ["W_c", f"{weight_changes_epoch['W_c']:.6f}"],
        ["W_o", f"{weight_changes_epoch['W_o']:.6f}"],
        ["W_y", f"{weight_changes_epoch['W_y']:.6f}"]
    ]
    print(tabulate(weight_table, headers=["Weight Matrix", "Change Magnitude"], tablefmt="grid"))

# After all epochs, print the summary of training
print("\n" + "=" * 80)
print("TRAINING SUMMARY")
print("=" * 80)

# Loss progression
print("\nLoss Progression:")
loss_table = []
for epoch, loss in enumerate(epoch_losses):
    loss_table.append([epoch+1, f"{loss:.6f}"])
print(tabulate(loss_table, headers=["Epoch", "Loss"], tablefmt="grid"))

# Weight change progression
print("\nWeight Change Magnitudes:")
weight_change_table = []
for epoch, changes in enumerate(weight_changes):
    weight_change_table.append([
        epoch+1, 
        f"{changes['W_f']:.6f}",
        f"{changes['W_i']:.6f}",
        f"{changes['W_c']:.6f}",
        f"{changes['W_o']:.6f}",
        f"{changes['W_y']:.6f}"
    ])
print(tabulate(weight_change_table, 
              headers=["Epoch", "W_f", "W_i", "W_c", "W_o", "W_y"], 
              tablefmt="grid"))

# Weight comparison (initial vs final)
print("\nWeight Comparison (Initial vs Final):")

# Calculate relative change in each weight matrix
def relative_change(initial, final):
    return np.linalg.norm(final - initial) / np.linalg.norm(initial) * 100

weight_comparison = [
    ["W_f", f"{np.linalg.norm(W_f_initial):.6f}", f"{np.linalg.norm(W_f):.6f}", 
     f"{relative_change(W_f_initial, W_f):.2f}%"],
    ["W_i", f"{np.linalg.norm(W_i_initial):.6f}", f"{np.linalg.norm(W_i):.6f}", 
     f"{relative_change(W_i_initial, W_i):.2f}%"],
    ["W_c", f"{np.linalg.norm(W_c_initial):.6f}", f"{np.linalg.norm(W_c):.6f}", 
     f"{relative_change(W_c_initial, W_c):.2f}%"],
    ["W_o", f"{np.linalg.norm(W_o_initial):.6f}", f"{np.linalg.norm(W_o):.6f}", 
     f"{relative_change(W_o_initial, W_o):.2f}%"],
    ["W_y", f"{np.linalg.norm(W_y_initial):.6f}", f"{np.linalg.norm(W_y):.6f}", 
     f"{relative_change(W_y_initial, W_y):.2f}%"]
]
print(tabulate(weight_comparison, 
              headers=["Weight Matrix", "Initial Norm", "Final Norm", "Relative Change"], 
              tablefmt="grid"))

print("\nBackpropagation Through Time (BPTT) Key Insights:")
print("1. BPTT involves unfolding the network through time and computing gradients at each timestep")
print("2. The gradients from later timesteps flow back to earlier timesteps (temporal dependency)")
print("3. The cell state (C_t) acts as a memory mechanism, preserving information across timesteps")
print("4. The gates (f_t, i_t, o_t) control information flow and have their own gradient paths")
print("5. Gradient clipping helps prevent the exploding gradient problem in recurrent networks")
print("6. The learning process adjusts weights to minimize the prediction error over time")

LSTM BACKPROPAGATION THROUGH TIME (BPTT) DEMONSTRATION
Training on 8 sequences with lookback=2 for 3 epochs
Learning rate: 0.01

============================== EPOCH 1 ==============================

--- FORWARD PASS: Sequence 1 (Days 1-2) ---
  Processed Day 1 - Hidden state norm: 0.1849
  Processed Day 2 - Hidden state norm: 0.2504
  Prediction for Day 3: 0.0028 kWh (Actual: 34.4384 kWh)

--- FORWARD PASS: Sequence 2 (Days 2-3) ---
  Processed Day 2 - Hidden state norm: 0.2774
  Processed Day 3 - Hidden state norm: 0.2853
  Prediction for Day 4: 0.0030 kWh (Actual: 48.3551 kWh)

--- FORWARD PASS: Sequence 3 (Days 3-4) ---
  Processed Day 3 - Hidden state norm: 0.2884
  Processed Day 4 - Hidden state norm: 0.2823
  Prediction for Day 5: 0.0031 kWh (Actual: 50.7292 kWh)

--- FORWARD PASS: Sequence 4 (Days 4-5) ---
  Processed Day 4 - Hidden state norm: 0.2798
  Processed Day 5 - Hidden state norm: 0.2622
  Prediction for Day 6: 0.0032 kWh (Actual: 58.0393 kWh)

--- FORWARD PASS: Sequen